# NGS pipeline for Alzheimer's Disease RNA-seq data

Pipeline to analyze RNA-seq data of an Alzheimer experiment. I have used 5 control samples and 5 Alzheimer samples.

In [1]:
import os
import pandas as pd
import csv
from IPython.display import FileLink
import matplotlib.pyplot as plt

%matplotlib inline

Initialize and setup some global variables:

In [11]:
# The git directory
os.environ['DIR'] = '/home/mees/NGS_Alzheimer/'
# Working directory to the sequencing data (I use an external disk, since I dont have a lot of space left on the pc)
os.environ['WORKDIR'] = '/media/mees/Elements/ngs_data/'
# The directory to the downloaded genome
os.environ['GENOMEDIR'] = '/media/mees/Elements/ngs_data/genomes/'
# Location to save the indexed reference genome
os.environ['REF_GENOME'] = '/media/mees/Elements/ngs_data/genomes/ref_genome/'
# FastQC perl file (without extension)
os.environ['FASTQC'] = '/home/mees/NGS/FastQC/fastqc'
# Hisat2-build file (without extension) or command
os.environ['HISAT_BUILD'] = '/home/mees/NGS_Alzheimer/hisat2-2.2.0/hisat2-build'
# Hisat2 file (without extension) or command
os.environ['HISAT'] = '/home/mees/NGS_Alzheimer/hisat2-2.2.0/hisat2'
# Samtools file (without extension) or command
os.environ['SAMTOOLS'] = 'samtools'
# Directory for logs
os.environ['LOGS'] = '/media/mees/Elements/ngs_data/logs/'
!mkdir -p $LOGS

## Download Data

First download the human genome for reference and save it as genome.fa in $GENOMEDIR. (-q, so that wget does not output anything).

Second, download the human gene transfer file for feature counting as genome.gtf in $GENOMEDIR.

Third, download the sample data via SRA tools. This process is found in download_data.sh. This data is download in $WORKDIR.

In [6]:
!wget -q ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
!gzip -d Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
!mkdir -p $GENOMEDIR
!mv Homo_sapiens.GRCh38.dna.primary_assembly.fa $GENOMEDIR/genome.fa

In [7]:
!wget -q ftp://ftp.ensembl.org/pub/release-100/gtf/homo_sapiens/Homo_sapiens.GRCh38.100.gtf.gz
!gzip -d Homo_sapiens.GRCh38.100.gtf.gz
!mkdir -p $GENOMEDIR
!mv Homo_sapiens.GRCh38.100.gtf $GENOMEDIR/genome.gtf

In [9]:
!mkdir -p $WORKDIR
!./src/download_data.sh -d $WORKDIR > $LOGS/download_data_log.txt 2> $LOGS/download_data_error_log.txt

## Quality Control

The quality of the RNA-seq data is determined using FastQC, which creates HTML reports. The script that contains calls to FastQC is fastqc.sh.

U can specifiy -c to copy the final reports to another directory, for instance, I copy it to the git repository.

In [10]:
!mkdir -p data/FastQC

In [24]:
!./src/fastqc.sh -d $WORKDIR -l $FASTQC -c $DIR/data 2> $LOGS/FastQC_log.txt

^C
'/media/mees/Elements/ngs_data//SRR6145423/FastQC/SRR6145423.sra_fastqc.zip' -> '/home/mees/NGS_Alzheimer//data/FastQC/SRR6145423.sra_fastqc.zip'
'/media/mees/Elements/ngs_data//SRR6145423/FastQC/SRR6145423.sra_fastqc.html' -> '/home/mees/NGS_Alzheimer//data/FastQC/SRR6145423.sra_fastqc.html'


In [12]:
for file in os.listdir('data/FastQC'):
    if file.endswith(".html"):
        html_file = FileLink(os.path.join('data/FastQC', file))
        display(html_file)

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145434.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145433.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145425.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145427.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145426.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145424.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145435.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145436.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145423.sra_fastqc.html

/home/mees/NGS_Alzheimer/data/FastQC/SRR6145437.sra_fastqc.html

FastQC shows that there are Truseq adapter index 7 and NNNNNNN sequences.
We want to remove these sequences using cutadapt, however, this is still in progress at the bottom of this notebook.

## Alignment with Hisat2

First, hisat2 needs to build the reference genome using hisat2-build, which can be found in build.sh.

Then, the samples can be aligned using hisat2, which can be found in align.sh.

In [32]:
!./src/build.sh -g $GENOMEDIR/genome.fa -o $REF_GENOME -l $HISAT_BUILD > $LOGS/hisat_build_log.txt 2> $LOGS/hisat_build_error_log.txt

^C


In [33]:
!./src/align.sh -g $REF_GENOME -f $WORKDIR/ -l $HISAT > $LOGS/hisat_align_log.txt

^C
(ERR): hisat2-align died with signal 2 (INT) 


## Converting .sam to .bam
In order to view the alignment in IGV and run HTSeq, the created sam files should processed, which is performed with samtools in samtobam.sh.

1. First, the files must be converted to bam files.
2. Second, the bam files must be sorted.
3. Third, the sorted bam files must be indexed.

In [34]:
!./src/samtobam.sh -d $WORKDIR -l $SAMTOOLS > $LOGS/samtobam_log.txt

[main_samview] fail to read the header from "/media/mees/Elements/ngs_data//SRR6145423/Hisat2/SRR6145423.sam".
samtools sort: failed to read header from "/media/mees/Elements/ngs_data//SRR6145423/Hisat2/SRR6145423.bam"
^C


# Counting Gene Features

Now that the alignment is finished, we count reads that are aligned to genes using HTSeq-count, which can be found in htseq.sh. In this script, it is specified that the alignments are ordered on position and that it is unstranded.

In [36]:
!./src/htseq.sh -d $WORKDIR -g $GENOMEDIR/genome.gtf > $LOGS/htseq_log.txt 2> $LOGS/htseq_error_log.txt

^C


## Differential Expression Using DESeq2

Now let us use DESeq2 to determine the differential expression.

First, make a file containing all the necessary information for running DESeq2.

Then, the R-script (DESeq2.r) can be executed. In this R script, the data is also plotted in various figures, which can be found in Rplots.pdf.

In [61]:
# Directory where the alignment etc. data is stored
workdir = os.environ['WORKDIR']

# Location the the file with the metadata
metadata = 'metadata.txt'
        
df = pd.read_csv(metadata, sep=',')


# Create a dictionary containing the necessary information
file_info = {}

# Adding HTSeq data to the dictionary
for file in os.listdir(workdir):
    split = file.split('.')
    if len(split) == 1 and split[0].startswith('SRR'):
        condition = df[df['Run'] == split[0]]['study_group'].values[0]
        file_info[split[0]] = {}
        file_info[split[0]]['HTSeq'] = os.path.join(workdir, split[0], 'Hisat2/' + split[0] + '_HTScount.txt')
        file_info[split[0]]['condition'] = condition.replace(', ', '_')
        
# Write data to a csv file      
file_out = 'DESeq2_data_table.csv'
with open(file_out, 'w') as f:
    f.write('SRR,HTSeq,condition')
    
    for key in file_info.keys():
        f.write("\n" + key + ',' + file_info[key]['HTSeq'] + ',' + file_info[key]['condition'])

In [41]:
!mkdir -p $WORKDIR/DESeq
!R < ./src/DESeq2.r --no-save > /dev/null 2> /dev/null
!cp -p $WORKDIR/DESeq/output.csv $DIR/data/DESeq_output.csv
!cp -p $WORKDIR/DESeq/output_LFC.csv $DIR/data/DESeq_LFC_output.csv

^C
